In [18]:
%matplotlib widget
#%matplotlib inline


import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d 
from helita.vis import rh15d_vis

from scipy.integrate import quadrature
from scipy.integrate import cumtrapz

from pathlib import Path

rhRepoPath = Path("/Users/elka127/Documents/code/rh")
#set_matplotlib_formats('svg')
# plt.rc('legend', frameon=False)
# plt.rc('figure', figsize=(7, 7 / 1.75)) # Larger figure sizes
# plt.rc('font', size=12)

# With the Bifrost Snapshot or with existing atmosperes' models

```
# There are four 1D models of flares on M-dwarfs (/mn/stornext/u3/matsc/rh/Atmos/mflares/). 
# These are:

# radyn_out.F13_dpl_GRID_pt1s.ncdf
# tx.m2F12-37.max2F12_Ec37_d3.ncdf
# tx.m5F12-37.max5F12_Ec37_d3.ncdf
# tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf

# the three "tx" models all have a low cut-off energy of 37 keV and a power index of 3 
# (see Allred et al. 2015) 
# but vary in maximum beam energy: 2e12, 5e12 and 1e13 erg/cm^2/s. 

#The model from Kowalski et al. 2015 is radyn_out.F13_dpl_GRID_pt1s.ncdf.
```



In [ ]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/code/rh/Atmos/"


In [ ]:
# to read xarray.Dataset and see dimentions, coordinates, data variables and attributes 
import xarray
# atmos = xarray.open_dataset(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf")
atmos = xarray.open_dataset(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf")
atmos
# out = xarray.open_dataset("/Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5")
# out
#output_indata.hdf5 file.

#output_ray.hdf5 file.

In [ ]:
hf=h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf")
tg1t=np.transpose(hf['temperature'][0,:,0,:])

z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)
print(z1t.shape, tg1t.shape)

In [ ]:
%matplotlib widget


fig, axs=plt.subplots()
im=axs.matshow(z1t, aspect='auto', origin='lower',extent=[0,10,0,18.9])
cbar=plt.colorbar(im)
axs.set_aspect('equal')
# plt.axes().set_aspect('equal')
# plt.show(im)
axs.set_xlabel('time')
axs.set_ylabel('real height')


```
#netcdf4 to hdf5
```

In [39]:
%matplotlib widget
import matplotlib.pyplot as plt

hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
ed1t=np.transpose(hf['electron_density'][0,:,0,:])



fig, ax=plt.subplots()
# cbar=plt.colorbar()
# ax.plot(z1t[:,:],np.log10(tg1t[:,:]))
# temperature189 = []
# height189 = []
# n = 50
n = 52
# n = 189
colors = plt.cm.jet(np.linspace(0,1,n))
for i in range(n):
# for i in [10,11,12,13]:

    ax.plot(z1t[:,i],np.log10(tg1t[:,i]), color=colors[i])
    # ax.plot(z1t[:,i],(vz1t[:,i]), color=colors[i])
    # ax.plot(z1t[:,i],(np.log10(ed1t[:,i])), color=colors[i])


    line = ax.lines[i]
    #temperature189tmp = line.get_ydata()
    # temperature189.append(line.get_ydata())
    # height189.append(line.get_xdata())

# # fig.canvas.draw()
# cbar=plt.colorbar(ax)
plt.xlabel('Height (Mm)')



# plt.ylabel('Velocity (m/s)')
plt.xlim(-0.2,0.5)
# plt.ylim(-1e5,2e5)
# plt.ylabel('lg(Electron Density ())')
#plt.title('m2F12-37 Atmos’)
# plt.title('txmF13-37 Atmos’)
plt.ylabel('lg(Temperature (K))')
plt.savefig('temperature')
# plt.savefig('velocity')
# plt.savefig('eldensity')
# data_0 = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
print(tg1t.shape)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(249, 52)


In [ ]:
def findNearestValue(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def findNearestIndex(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

# print(z1t.min(), z1t.max())
lineyka = np.linspace(z1t.min(), z1t.max(), 249)
# print(lineyka)

vseStro4ki = []
vsegoVremen = z1t.shape[1]
for i in range(vsegoVremen):
    tmp = []   
    for ii in lineyka:
        nindex = findNearestIndex(z1t[i,:], ii)

        #print(tg1t[i,nindex])
        # tmp.append(tg1t[i,nindex])
        # tmp.append(vz1t[i,nindex])
        tmp.append(ed1t[i,nindex])

    vseStro4ki.append(tmp)

# temperatureF13 = np.array(vseStro4ki)
# velocityzF13 = np.array(vseStro4ki)
electrondF13 = np.array(vseStro4ki)

# print(velocityzF13.shape)

In [ ]:
%matplotlib widget

fig, axs = plt.subplots()


# im=axs.matshow(temperatureF13,aspect='auto', origin='lower',extent=[0,10,0,5],cmap='plasma')
# im=axs.matshow(velocityzF13,aspect='auto', origin='lower',extent=[0,10,0,5],cmap='inferno')
im=axs.matshow(np.log10(electrondF13),aspect='auto', origin='lower',extent=[0,10,0,5],cmap='magma')

cbar=plt.colorbar(im)
# cbar.set_label('Temperature, log10, K')
# cbar.set_label('Velocity Z, m$s^{-1}$')

cbar.set_label(f'Electron Density, log10, m$^{-3}$')

# ax.set_xlim(10)
# ax.set_ylim(18.9)
axs.set_xlabel('Height (Mm)')
axs.set_ylabel('Time, s')

In [ ]:
%matplotlib widget

fig, axs = plt.subplots()


im=axs.matshow(hgt, aspect='auto', origin='lower',extent=[0,10,0,18.9])
cbar=plt.colorbar(im)
cbar.set_label('Temperature, log10, K')
# ax.set_xlim(10)
# ax.set_ylim(18.9)
axs.set_xlabel('Height (Mm)')
axs.set_ylabel('Time, s')

In [ ]:
%matplotlib widget

fig, axs = plt.subplots()

z = temperature
nx, ny = (249, 189)
x = np.linspace(0, 10, nx)
y = np.linspace(0, 189, ny)
xv, yv = np.meshgrid(x, y)

# xi = np.linspace(1,6,1)
# yi = np.linspace(2,16,2)
plt.contourf(xv,yv,z)

In [ ]:
atmos.to_netcdf("newfile.hdf5", format='NETCDF4')

```
# виджет хелиты все равно не работает даже с переделанным файлом из netcdf
# строит только 1 график
```

# Plotting atmospheres in real height

In [ ]:
%matplotlib widget


# hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m5F12-37.max2F12_Ec37_d3.ncdf","r")
hf=h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
ed1t=np.transpose(hf['electron_density'][0,:,0,:])

z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

fig, axs=plt.subplots()

axs.plot(z1t[:,0],np.log10(tg1t[:,120]),label='12.0 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,75]),label='7.5 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,60]),label='6.0 sec')

axs.plot(z1t[:,0],np.log10(tg1t[:,40]),label='4.0 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,22]),label='2.2 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,12]),label='1.2 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,2]),label='0.2 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,1]),label='0.1 sec')


# ax.plot(z1t[:,0],np.log10(tg1t[:,0]),'g-',label='0 sec')

# ax.plot(z1t[:,0],vz1t[:,135],label='13.5 sec')
# ax.plot(z1t[:,0],vz1t[:,90],label='9.0 sec')
# ax.plot(z1t[:,0],vz1t[:,60],label='6.0 sec')

# ax.plot(z1t[:,0],vz1t[:,40],label='4.0 sec')
# ax.plot(z1t[:,0],vz1t[:,22],label='2.2 sec')
# ax.plot(z1t[:,0],vz1t[:,12],label='0.2 sec')
# ax.plot(z1t[:,0],vz1t[:,2],label='0.2 sec')

# ax.plot(z1t[:,0],vz1t[:,1],label='0.1 sec')
# ax.plot(z1t[:,0],vz1t[:,0],label='0 sec')

# ax.plot(z1t[:,0],np.log10(ed1t[:,130]),label='13.0 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,100]),label='10.0 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,75]),label='7.5 sec')

# ax.plot(z1t[:,0],np.log10(ed1t[:,40]),label='4.0 sec')
#ax.plot(z1t[:,0],np.log10(ed1t[:,22]),label='2.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,12]),label='1.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,2]),label='0.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,0]),label='0 sec')




plt.xlabel('Height (Mm)')

plt.ylabel('lg(Temperature (K))')
# plt.ylabel('Velocity (m/s)')
# plt.xlim(-0.2,0.5)
# plt.ylim(-1e5,2e5)
#plt.ylabel('lg(Electron Density ())')
#plt.title('m2F12-37 Atmos’)
# plt.title('txmF13-37 Atmos’)

axs.legend()  # Add a legend.


plt.savefig("temperature_.svg", format="svg")
# plt.savefig("velocity_z_.svg", format="svg")
# plt.savefig("electron_density_.svg", format="svg")


# Create own wavetable file

```
# Writes RH wave file (in xdr format). All wavelengths should be in nm.

# Parameters
# ----------
# start: number
#     Starting wavelength.
# end: number
#     Ending wavelength (non-inclusive)
# step: number
#     Wavelength separation
# outfile: string
#     Name of file to write.
# ewave: 1-D array, optional
#    Array of existing wavelengths. Program will make discard points
#    to make sure no step is enforced using these points too.
# air: boolean, optional
#     If true, will at the end convert the wavelengths into vacuum
#     wavelengths.
```

In [ ]:
def make_wave_file(my_wave, start=20, end=4020, step=1.5, new_wave=None,
                   ewave=None, air=False):
                   rh15d.make_wave_file('my_wave.wave', ewave=my_waves)

In [ ]:
rh15d.make_wave_file('My_wave.wave', 20, 4020, 1.5)

In [ ]:
#from helita.sim import rh15d
#read exisitng file if needed
MY_wave = rh15d.read_wave_file(rhRepoPath / "Atoms/wave_files/My_wave.wave")


# Intencity plot for Ca II

In [19]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:40: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  setattr(self, g, xr.open_dataset(infile, group=g, autoclose=True))
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:53: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  self.ray 

In [ ]:
%matplotlib widget
#%matplotlib inline
data.ray.intensity.plot()
#plt.legend(loc="upper left")
plt.xlabel("Wavelenght nm")
plt.ylabel("Time, s*10")
#plt.axis([20, 1000, 0, 62])

```
# to see what's in the output data
```

In [ ]:
hf=h5py.File(rhRepoPath / "rh15d/run/output/output_ray.hdf5","r")

hf.keys()
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
# x = data.ray.x
# # tau1.plot()
print(data.ray.x.shape)

# Tau=1 height

In [ ]:
%matplotlib widget

fig, ax=plt.subplots()

ax.plot(wave[indices],data.ray.tau_one_height[0,0,indices]/1e6, 'k--',label='0 sec')
ax.plot(wave[indices],data.ray.tau_one_height[2,0,indices]/1e6, 'r-',label='0.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[12,0,indices]/1e6, 'r-.',label='1.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[22,0,indices]/1e6, 'b--',label='2.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[40,0,indices]/1e6, 'b-.',label='4. sec')

ax.legend(loc="upper right") 
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Height (Mm)')
plt.title(r'$\tau=1$')
plt.savefig("Tau_one_height_.svg", format="svg")


In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")


indices = np.arange(len(data.ray.wavelength))
#print(indices)
wave.sel(wavelength=91, method="nearest")
index91 = np.argmin(np.abs(wave.data - 91))

wave.sel(wavelength=123, method="nearest")
index123 = np.argmin(np.abs(wave.data - 123))

wave.sel(wavelength=393, method="nearest")
index393 = np.argmin(np.abs(wave.data - 393))

wave.sel(wavelength=854, method="nearest")
index854 = np.argmin(np.abs(wave.data - 854))

# index500 = np.argmin(np.abs(wave.data - 500))
f = open(rhRepoPath / "rh15d/run/ray.input", "w")  # this will overwrite any existing file!
f.write('1.00\n')
output = str(len(indices) + 4)
# ololo = 0
for ind in indices:
    
    output += f" {ind}"
output += f" {index91}\n"
output += f" {index123}\n"
output += f" {index393}\n"
output += f" {index854}\n"


f.write(output)
f.close()




In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 200) & (wave < 500)]
#If we want to see also one particular wavelength (500 in example)
wave.sel(wavelength=500, method="nearest")
index500 = np.argmin(np.abs(wave.data - 500))

with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:
    f.write("1.00\n")
    output = str(len(indices) + 1)
    for ind in indices:
        output += f" {ind}"
    output += f" {index500}\n"
    f.write(output)

# Writing ray.input file
```
# Selecting wavelengths for detailed output
# Choosing all wavelenghts range
# Ray input includes all indexes
```

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
indices = np.arange(len(data.ray.wavelength))
with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:

    f.write('1.00\n')
    output = str(len(indices)+1)
    for ind in indices:
    
        output += f" {ind}"
    f.write(output)
f.close()

```
#Need to run RH15D again with a new ray.input file 

# Calculating Flux from Intensity(mu)

```
# reading new output
# writing all tau (time) in one array
```

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

indices = np.arange(len(data.ray.wavelength))
print(len(indices))
print(data.atmos.height_scale.shape)

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
indices = np.arange(len(data.ray.wavelength))
aar = [0,2,12,22,40]
for a in aar:
    height = data.atmos.height_scale[a, 0].dropna('height')  
    
    index = []
    for i in indices:
        idx = np.argmin(np.abs(data.ray.wavelength.data - data.ray.wavelength[i].data))
        index.append(idx)
    massivTau = []
    for i in indices:
        x = cumtrapz(data.ray.chi[a, 0, :, i].dropna('height'), x=-height)
        x = np.concatenate([[1e-20], x])
        massivTau.append(x)
    np.save(f"massivTau-{a}", massivTau)


```
# calculating I(\mu) for 5 \mu
```

In [ ]:
from scipy import integrate

sourceFunction = data.ray.source_function

mu = [0.0469101,0.230765,0.500000,0.769235,0.953090]
weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]
massivTau = np.load("massivTau-12.npy")
# intensityMu =[]
for m in range(5):
    intensityMu = []

    for y in indices:
    
        tmpMu = 0
        # for i in range(len(massivTau[y])):
        # x = np.log(massivTau[y])
        
        x = massivTau[y]
        # x1 = np.log10(massivTau[y])
        # x = np.log(x)
        # tmpMu = integrate.trapz(sourceFunction[0,0,:,y].data * np.exp(-x[:]/mu), x)
        # tmpMu = integrate.trapz(sourceFunction[120,0,:,y].data * np.exp(-x[:]/mu[m]), x) 
        tmpMu = integrate.simps(sourceFunction[12,0,:,y].data * np.exp(-x[:]/mu[m]), x)
        # tmpMu = np.power(tmpMu, 10)
        
        intensityMu.append(1/mu[m] * tmpMu)
        # intensityMu.append(tmpMu)
    np.save(f"intensityMu-{m+1}.npy", intensityMu)

    print(f"Всего элементоу: {len(intensityMu)}")
    print(f"Перший элемент: {intensityMu[0]}")
    print(f"Второй элемент: {intensityMu[1]}")
    print(f"Последний элемент: {intensityMu[1643]}")

# ---

# fig, ax = plt.subplots()

# ax.plot(data.atmos.height_scale[2,0]/1e6,sourceFunction[2,0,:,0],label='0.2 sec')
#ax.plot(np.log10(massivTau[792]),sourceFunction[22,0,:,0],label='0.2 sec')


In [ ]:
%matplotlib widget

wave = data.ray.wavelength
# ImuLoaded = np.load("IntensityMu12.npy")

ImuLoaded1 = np.load("IntensityMu-1.npy")
ImuLoaded2 = np.load("IntensityMu-2.npy")
ImuLoaded3 = np.load("IntensityMu-3.npy")
ImuLoaded4 = np.load("IntensityMu-4.npy")
ImuLoaded5 = np.load("IntensityMu-5.npy")
# # print(ImuLoaded1.shape)
fig, ax=plt.subplots()
# #np.load("Flux.npy")



# ax.plot(wave[indices], ImuLoaded,'k--',label=r'$\mu$1')

# ax.plot(wave[indices], ImuLoaded5,'r-',label=r'$\mu$0.95')
# ax.plot(wave[indices],data.ray.intensity[12,0,indices],'g-.',label='1.2 sec')

ax.plot(wave[indices], ImuLoaded1,'k--',label=r'$\mu$1')
ax.plot(wave[indices], ImuLoaded2,'k-',label=r'$\mu$2')
ax.plot(wave[indices], ImuLoaded3,'g-',label=r'$\mu$3')
ax.plot(wave[indices], ImuLoaded4,'r-',label=r'$\mu$4')
ax.plot(wave[indices], ImuLoaded5,'b-',label=r'$\mu$5')

ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Intensity (Stokes I) ($W/(m^2*Hz*sr)$')
#ax.set_yscale('log')
#plt.axis([20, 4000, 0, 8e-8])
plt.savefig("Intencity_.svg", format="svg")

In [ ]:
from scipy import integrate


   
sourceFunction = data.ray.source_function
massivTau = np.load("massivTau-0.npy")

# a = 0.4342944819 #(log_10(e))
# mu = [0.0469101,0.230765,0.500000,0.769235,0.953090]
# weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]
# mu = 1.0
intensityMu =[]

for y in indices: 
    tmpMu = 0
    # x = np.log10(massivTau[y])
    # print(x)
    x = massivTau[y]
    # tmpMu = integrate.simps(sourceFunction[0,0,:,y].data * np.exp(-x[:]), np.log10(x))
        # tmpMu = np.power(tmpMu, 10)
    
    tmpMu = integrate.trapz(np.log(sourceFunction[0,0,:,y].data * np.exp(-(x[:]))),np.log(x))
    # tmpMu = integrate.trapz(sourceFunction[120,0,:,y].data * np.exp(-(x[:])),np.log10(x))
    # print(tmpMu)
    # intensityMu.append(tmpMu*(np.power(10,np.log10(tmpMu))))
    # print(10**tmpMu,"\n",tmpMu)
    # intensityMu.append(tmpMu)
    # print(tmpMu)
    # print('---')
    # print(-np.log10(tmpMu))
    intensityMu.append(tmpMu)
    # intensityMu.append(tmpMu)
    #2.71828182846e+9
print(intensityMu[0],"\n",intensityMu[1],"\n",intensityMu[93],"\n", intensityMu[387],"\n"   )
np.save(f"Intensity1Mu0.npy", intensityMu)


In [ ]:
%matplotlib widget
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

wave = data.ray.wavelength
ImuLoaded = np.load("Intensity1Mu0.npy")

# ImuLoaded1 = np.load("IntensityMu-1.npy")
# ImuLoaded2 = np.load("IntensityMu-2.npy")
# ImuLoaded3 = np.load("IntensityMu-3.npy")
# ImuLoaded4 = np.load("IntensityMu-4.npy")
# ImuLoaded5 = np.load("IntensityMu-5.npy")
# # print(ImuLoaded1.shape)
fig, ax=plt.subplots()
# #np.load("Flux.npy")



ax.plot(wave[indices],ImuLoaded,'k--',label=r'$\mu$1')

# ax.plot(wave[indices], ImuLoaded5,'r-',label=r'$\mu$0.95')
ax.plot(wave[indices],data.ray.intensity[0,0,indices],'g-.',label='12 sec')

# ax.plot(wave[indices], ImuLoaded1,'k--',label=r'$\mu$1')
# ax.plot(wave[indices], ImuLoaded2,'k-',label=r'$\mu$2')
# ax.plot(wave[indices], ImuLoaded3,'g-',label=r'$\mu$3')
# ax.plot(wave[indices], ImuLoaded4,'r-',label=r'$\mu$4')
# ax.plot(wave[indices], ImuLoaded5,'b-',label=r'$\mu$5')

ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Intensity (Stokes I) ($W/(m^2*Hz*sr)$')
#ax.set_yscale('log')
# plt.axis([20, 4000, 0, 6e-8])
plt.savefig("Intencity_.svg", format="svg")

In [ ]:
fluxesImu =[]
weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]
mu = [0.0469101,0.230765,0.500000,0.769235,0.953090]
ImuLoaded1 = np.load("IntensityMu-1.npy")
ImuLoaded2 = np.load("IntensityMu-2.npy")
ImuLoaded3 = np.load("IntensityMu-3.npy")
ImuLoaded4 = np.load("IntensityMu-4.npy")
ImuLoaded5 = np.load("IntensityMu-5.npy")
Imuloaded = []
Imuloaded.append(ImuLoaded1)
Imuloaded.append(ImuLoaded2)
Imuloaded.append(ImuLoaded3)
Imuloaded.append(ImuLoaded4)
Imuloaded.append(ImuLoaded5)
#print(len(Imuloaded))
for i in indices:
    imu = 0
    for i5 in range(len(weightMu)):
        imu += Imuloaded[i5][i]*weightMu[i5]*mu[i5]
    fluxesImu.append(2*np.pi*imu)
print(len(fluxesImu))


In [ ]:
np.save("fluxesImu120.npy", fluxesImu)

In [ ]:
%matplotlib widget

fluxesImu1 = np.load("fluxesImu0.npy")
fluxesImu2 = np.load("fluxesImu2.npy")
fluxesImu3 = np.load("fluxesImu12.npy")
fluxesImu4 = np.load("fluxesImu22.npy")
fluxesImu5 = np.load("fluxesImu40.npy")
# fluxesImu6 = np.load("fluxesImu60.npy")
# fluxesImu7 = np.load("fluxesImu120.npy")



fig, ax=plt.subplots()
#np.load("Flux.npy")

wave = data.ray.wavelength

ax.plot(wave[indices], fluxesImu1,'k--',label='0 sec')
ax.plot(wave[indices], fluxesImu2,'k-',label='0.2 sec')
ax.plot(wave[indices], fluxesImu3,'r-',label='1.2 sec')
ax.plot(wave[indices], fluxesImu4,'b-',label='2.2 sec')
ax.plot(wave[indices], fluxesImu5,'g-',label='4.0 sec')
# ax.plot(wave[indices], fluxesImu6,'y-',label='6.0 sec')
# ax.plot(wave[indices], fluxesImu7,'c-',label='12.0 sec')


ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Flux ($W/(m^2*Hz)$')
#ax.set_yscale('log')
#plt.axis([20, 4000, 0, 8e-8])
plt.savefig("FluxperHz_.svg", format="svg")
# ax.plot(wave[indices], ImuLoaded5)
# plt.xlabel("Wavelenght, nm")
# plt.ylabel("Flux, W/cm^2")
#plt.xlim(200,1000)
# plt.axis([2000, 10000, 0, 1e8])
#plt.axis([200, 1000,0, 1e-7])


In [ ]:
const1 = 2.99792458e+21
fluxesImu = np.load("fluxesImu120.npy")
fluxesSurfaceMu = []
# fluxesSurfaceMu.append(const1 * fluxesImu[0] /(np.power(wave[0] * 10,2)))
# print(fluxesSurfaceMu)
for i in indices:
    fluxesSurfaceMu.append(const1 * fluxesImu[i] /(np.power(wave[i] * 10,2)))

np.save("fluxesSurfaceMu120.npy", fluxesSurfaceMu)

```
# calculating total flux and effective temperature
```

In [ ]:
from scipy import integrate
fluxesSurfaceMu = np.load("fluxesSurfaceMu0.npy")

x = wave[indices]
fluxesSum = integrate.trapz(fluxesSurfaceMu[:], x * 10)
print(fluxesSum)

In [ ]:
sigma = 5.6784e-8
Teff = np.power(fluxesSum*1e-3/sigma, 1/4)
print(Teff)

In [ ]:
fig, ax=plt.subplots()

ax.plot(wave[indices], fluxesSurfaceMu)
plt.xlabel("Wavelenght (nm)")
plt.ylabel("Flux")
# plt.xlim(200,1000)
# plt.ylim(0,8e8)


In [ ]:
fluxesSurfaceMu1 = np.load("fluxesSurfaceMu0.npy")
fluxesSurfaceMu2 = np.load("fluxesSurfaceMu2.npy")
fluxesSurfaceMu3 = np.load("fluxesSurfaceMu12.npy")
fluxesSurfaceMu4 = np.load("fluxesSurfaceMu22.npy")
fluxesSurfaceMu5 = np.load("fluxesSurfaceMu40.npy")
# fluxesSurfaceMu6 = np.load("fluxesSurfaceMu60.npy")
# fluxesSurfaceMu7 = np.load("fluxesSurfaceMu120.npy")



fig, ax=plt.subplots()

ax.plot(wave[indices]*10, fluxesSurfaceMu1,'k--',label='0 sec')
ax.plot(wave[indices]*10, fluxesSurfaceMu2,'k-',label='0.2 sec')
ax.plot(wave[indices]*10, fluxesSurfaceMu3,'g--',label='1.2 sec')
ax.plot(wave[indices]*10, fluxesSurfaceMu4,'g-',label='2.2 sec')
ax.plot(wave[indices]*10, fluxesSurfaceMu5,'r--',label='4.0 sec')
# ax.plot(wave[indices]*10, fluxesSurfaceMu6,'b--',label='6.0 sec')
# ax.plot(wave[indices]*10, fluxesSurfaceMu7,'b-',label='12.0 sec')


ax.legend()
plt.xlabel("Wavelenght, Å")
plt.ylabel("Flux, erg/cm^2/s/Å")
# plt.axis([2000, 9000, 0, 7e8])
plt.axis([0, 10000, 0, 4.55e9])

plt.savefig("flux_.svg", format="svg")

```
# flux (lambda) normilized by continuum
# The continuum-normalised spectrum is derived by dividing the computed flux 
# spectrum by the computed continuum flux in every spectral bin. 
```

In [ ]:
fluxesSur1 = np.load("fluxesImu0.npy")
fluxesSur2 = np.load("fluxesImu2.npy")
fluxesSur3 = np.load("fluxesImu12.npy")
fluxesSur4 = np.load("fluxesImu22.npy")
fluxesSur5 = np.load("fluxesImu40.npy")
# fluxesSur6 = np.load("fluxesImu60.npy")
# fluxesSur7 = np.load("fluxesImu120.npy")

FluxHloaded = []
FluxHloaded.append(fluxesSur1)
FluxHloaded.append(fluxesSur2)
FluxHloaded.append(fluxesSur3)
FluxHloaded.append(fluxesSur4)
FluxHloaded.append(fluxesSur5)
# FluxHloaded.append(fluxesSur6)
# FluxHloaded.append(fluxesSur7)

for i in range(1,5):
    normFlux = FluxHloaded[i]/FluxHloaded[0]
    # normFluxTocontinuum = (normFlux - normFlux[np.argmin(normFlux)])/(normFlux[np.argmax(normFlux)] - normFlux[np.argmin(normFlux)])
    np.save(f"normFluxSurCon{i+1}.npy", normFlux)

In [ ]:
normFluxCon2 = np.load("normFluxSurCon2.npy")
normFluxCon12 = np.load("normFluxSurCon3.npy")
normFluxCon22 = np.load("normFluxSurCon4.npy")
normFluxCon40 = np.load("normFluxSurCon5.npy")
# normFluxCon60 = np.load("normFluxSurCon6.npy")
# normFluxCon120 = np.load("normFluxSurCon7.npy")

fig, ax=plt.subplots()

wave = data.ray.wavelength


ax.plot(wave[indices], normFluxCon2,label='0.2 sec')
ax.plot(wave[indices], normFluxCon12,label='1.2 sec')
ax.plot(wave[indices], normFluxCon22,label='2.2 sec')
ax.plot(wave[indices], normFluxCon40,label='4.0 sec')
# ax.plot(wave[indices], normFluxCon60,label='6.0 sec')
# ax.plot(wave[indices], normFluxCon120,label='12.0 sec')


ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Normalized Flux')
# #ax.set_yscale('log')
plt.axis([0, 800, -10000, 1.3e5])
plt.savefig("FluxnormSur_.svg", format="svg")

```
#To make a plot for intensity at a fixed wavelength
```

In [ ]:
%matplotlib widget
data.ray.intensity.sel(wavelength=500, method='nearest').plot()

# Tau=1 for wavelenght

In [ ]:
%matplotlib widget

#data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")


fig, ax = plt.subplots()
ax.plot(height / 1e6, massivTau[20])  # height in Mm

plt.ylabel(r'$\tau$$_{20}$')
plt.xlabel('H (Mm)')

ax.set_yscale('log')
ax.axhline(y=1,linestyle='dashed',color='black')
#ax.axvline(x=0.5,linestyle='dashed',color='black')
plt.savefig("tau360_.svg", format="svg")


# Plot the departure coefficients for the levels of Ca II



In [ ]:
%matplotlib widget
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

plt.figure()
for i in range(5):
    plt.plot(data.atmos.height_scale[22,0]/1e6, 
             data.atom_CA.populations[i, 22, 0]/data.atom_CA.populations_LTE[i, 22, 0],
             label='Level %i' % (i + 1))
plt.legend(loc="upper left")
plt.ylabel("Departure coefficients at 0 sec")
plt.xlabel("H (Mm)")
#plt.axis([0, 0.4])

# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
# fig, ax = plt.subplots()
# ax.plot(tau500,data.atom_CA.populations[1, 0, 0, 0]/data.atom_CA.populations_LTE[1, 0, 0, 0], label='Level %1' % (1)) 
# plt.legend(loc="upper left")
# plt.xlabel("tau500")
# plt.ylim(0.05, 2)
# plt.ylabel("Departure coefficients")
# plt.ylim(0.05, 2)
plt.axis([0, 0.5, 0, 1e5])


In [ ]:
%matplotlib widget

fig, ax = plt.subplots()
ax.plot(data.atmos.height_scale[0, 0].dropna('height')/1e6, data.atom_CA.populations[0,0,0,:].dropna('height')/data.atom_CA.populations_LTE[0,0,0,:].dropna('height'),'r--',label=r'b$_{l}$')
ax.plot(data.atmos.height_scale[0, 0].dropna('height')/1e6, np.log10(data.atom_CA.populations[0,3,0,:].dropna('height')/data.atom_CA.populations_LTE[0,3,0,:].dropna('height')),'k-',label=r'b$_{u}$')
plt.legend(loc='best')
plt.xlabel('H (Mm)')
plt.ylabel(r'Log (b$_{i}$)')
#plt.xlim([0,2])

# tau=1

In [20]:
%matplotlib widget

data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(data.ray.wavelength))
print(data.ray.tau_one_height.shape)
fig, ax = plt.subplots()

ax.plot(wave[indices],data.ray.tau_one_height[0,0,indices]/1e6,'k--',label='0 sec')
ax.plot(wave[indices],data.ray.tau_one_height[2,0,indices]/1e6,'r-',label='0.2 sec')
#ax.plot(wave[indices],data.ray.tau_one_height[12,0,indices]/1e6,'r-',label='1.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[22,0,indices]/1e6,'b-',label='2.2 sec')
#ax.plot(wave[indices],data.ray.tau_one_height[40,0,indices]/1e6,'g-',label='4 sec')

plt.legend(loc='best')

plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Height (Mm)')


--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
(50, 1, 1644)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Height (Mm)')

In [ ]:
height = data.ray.height
fig, ax = plt.subplots()
ax.plot(height/)


# от тау 500

In [ ]:
# xVals = data.atmos.height_scale[0,0]/1e6
# with open("/tmp/x.txt", "w") as f:
#     for x in xVals.data:
#         f.write(f"{x}\n")

print(data.atom_CA.populations[0,22,0])
# yVals = data.atom_CA.populations[0,0,0]
# with open("/tmp/y.txt", "w") as f:
#     for y in yVals.data:
#         f.write(f"{y}\n")


#print(data.atom_CA.populations[0,0,0][247])
# print("- - - - -")
# print(data.atom_CA.populations[1,0,0])
# print("- - - - -")
# print(data.atom_CA.populations[2,0,0])

In [ ]:
import matplotlib.animation as animation

fig, ax = plt.subplots()

line, = ax.plot(
    data.atmos.height_scale[0,0]/1e6,
    data.atom_CA.populations[0,0,0]
)

def animate(i):
    #line.set_xdata(x)
    line.set_ydata(data.atom_CA.populations[0, i, 0][0])
    return line,

ani = animation.FuncAnimation(
    fig,
    animate,
    np.arange(52),
    #blit=True,
    repeat=False
)
plt.show()

In [ ]:
%matplotlib widget

plt.figure()
for i in range(5):
    plt.plot(
        data.atmos.height_scale[22,0]/1e6, 
        data.atom_CA.populations[i, 22, 0]/data.atom_CA.populations_LTE[i, 22, 0],
        label='Level %i' % (i + 1)
    )
plt.legend(loc="upper left")
plt.xlabel("Height (Mm)")
plt.ylabel("Departure coefficients")
plt.xlim(0,4)
plt.ylim(-20,1e6)
#/data.atom_CA.populations_LTE[i, 100, 0]

# Departure coefficient vs optical depth

In [ ]:
%matplotlib widget

for i in range(6):
    plt.plot(np.log10(tau669), np.log10(data.atom_CA.populations[i,22,0,:].dropna('height')/data.atom_CA.populations_LTE[i,22,0,:].dropna('height')),label=r'b'+r'$_{'+str(i)+'}$')
    plt.legend(loc='best')
    plt.xlabel(r'Log ($\tau$$_{669}$)')
    plt.ylabel(r'Log (b$_{i}$)')
    plt.xlim([-12,2])

# Source function

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

rh15d_vis.SourceFunction(data);
plt.xlim(-0.2,0.4)
plt.ylim(0,1e6)

In [ ]:
%matplotlib widget

data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
hf=h5py.File(rhRepoPath / "rh15d/run/output/output_ray.hdf5","r")

hf.keys()

sourceFunction = hf['source_function']

fig, ax = plt.subplots()

# ax.plot(data.atmos.height_scale[2,0]/1e6,sourceFunction[2,0,:,0],label='0.2 sec')
ax.plot(np.log10(tau669),sourceFunction[2,0,:,0],label='0.2 sec')

#data.source_function.data

# Calculating Normalized Intesity

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
# /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5
hf=h5py.File(rhRepoPath / "rh15d/run/output/output_ray.hdf5","r")

hf.keys()

wave = data.ray.wavelength

print(data.ray.intensity.shape)


In [ ]:
%matplotlib widget
Intencity1 = data.ray.intensity[0,0,:]
Intencity2 = data.ray.intensity[2,0,:]
Intencity3 = data.ray.intensity[12,0,:]
Intencity4 = data.ray.intensity[22,0,:]
Intencity5 = data.ray.intensity[40,0,:]
# Intencity6 = data.ray.intensity[60,0,:]
# Intencity7 = data.ray.intensity[120,0,:]

fig, ax = plt.subplots()
ax.plot(wave[indices],Intencity1,label='0 sec')
ax.plot(wave[indices],Intencity2,label='0.2 sec')
ax.plot(wave[indices],Intencity3,label='1.2 sec')
ax.plot(wave[indices],Intencity4,label='2.2 sec')
ax.plot(wave[indices],Intencity5,label='4 sec')
# ax.plot(wave[indices],Intencity6,label='6 sec')
# ax.plot(wave[indices],Intencity7,label='12 sec')


plt.legend(loc='best')
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Intensity (Stokes I) ($W/(m^2*Hz*sr)$')

plt.savefig("Intensitym1_.svg", format="svg")


In [ ]:


Intencityloaded = []
Intencityloaded.append(Intencity1)
Intencityloaded.append(Intencity2)
Intencityloaded.append(Intencity3)
Intencityloaded.append(Intencity4)
Intencityloaded.append(Intencity5)
# Intencityloaded.append(Intencity6)
# Intencityloaded.append(Intencity7)

for i in range(1,5):
    normIntencity = Intencityloaded[i]/Intencityloaded[0]
    # normFluxTocontinuum = (normFlux - normFlux[np.argmin(normFlux)])/(normFlux[np.argmax(normFlux)] - normFlux[np.argmin(normFlux)])
    np.save(f"normIntencityCon{i+1}.npy", normIntencity)

In [ ]:
%matplotlib widget
# normIntencityCon1 = np.load("normIntencityCon0.npy")
normIntencityCon2 = np.load("normIntencityCon2.npy")
normIntencityCon3 = np.load("normIntencityCon3.npy")
normIntencityCon4 = np.load("normIntencityCon4.npy")
normIntencityCon5 = np.load("normIntencityCon5.npy")
# normIntencityCon6 = np.load("normIntencityCon6.npy")
# normIntencityCon7 = np.load("normIntencityCon7.npy")

fig, ax = plt.subplots()
# ax.plot(indices,normIntencityCon1,label='0 sec')
ax.plot(wave[indices],normIntencityCon2,label='0.2 sec')
ax.plot(wave[indices],normIntencityCon3,label='1.2 sec')
ax.plot(wave[indices],normIntencityCon4,label='2.2 sec')
ax.plot(wave[indices],normIntencityCon5,label='4 sec')
# ax.plot(wave[indices],normIntencityCon6,label='6 sec')

# ax.plot(wave[indices],normIntencityCon7,label='12 sec')


plt.legend(loc='best')
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Normalized Intensity (Stokes I)')
plt.axis([0, 800, -10000, 1.5e5])

plt.savefig("IntensityRH_.svg", format="svg")

# Total Flux in UV bands

In [ ]:
# calculating total flux and effective temperature
# first calculate Flux (lambda), then integrate for choosen wavelengt bands
from scipy import integrate
# wave = data.ray.wavelength

const1 = 2.99792458e+21
fluxesSurfaceMu = []
# fluxesImu = np.load("fluxesImu0.npy")
# fluxesImu  = np.load("fluxesImu2.npy")
# fluxesImu  = np.load("fluxesImu12.npy")
fluxesImu  = np.load("fluxesImu22.npy")
# fluxesImu  = np.load("fluxesImu40.npy")

selectedWave = []
# fluxesSurfaceMu.append(const1 * fluxesImu[0] /(np.power(wave[0] * 10,2)))
# print(fluxesSurfaceMu)
for i in indices:
    v = wave[i].data
    if v > 315 and v < 400:
        fluxesSurfaceMu.append(const1 * fluxesImu[i] /(np.power(wave[i].data * 10,2)))
        selectedWave.append(wave[i])
        print(v)    
fluxesSurfaceMuNP = np.array(fluxesSurfaceMu)
selectedWaveNP = np.array(selectedWave)
print(fluxesSurfaceMuNP.shape, selectedWaveNP.shape)

In [ ]:
fluxesSum = integrate.trapz(fluxesSurfaceMuNP[:], selectedWaveNP*10)
print(fluxesSum)

In [ ]:
sigma = 5.6784e-8
Teff = np.power(fluxesSum*1e-3/sigma, 1/4)
print(Teff)

# Calculating Flux changing in time for selected wavelenghts

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength

indices = np.arange(len(data.ray.wavelength))

aar = []
for j in indices:
    vT = wave[j].data
    if vT > 260 and vT < 280:
        aar.append(j)
        print(j)

for i in aar:
        
    massivTauTime = []
    for a in range(len(data.ray.x)):
        
        height = data.atmos.height_scale[a, 0].dropna('height')  
    
    # index = []
    # for i in indices:
    #     idx = np.argmin(np.abs(data.ray.wavelength.data - data.ray.wavelength[i].data))
    #     index.append(idx)
   
        x = cumtrapz(data.ray.chi[a, 0, :, i].dropna('height'), x=-height)
        x = np.concatenate([[1e-20], x])
        massivTauTime.append(x)
    np.save(f"massivTauTime-{i}", massivTauTime)
    # np.save(f"massivTauTime-{np.round(wave[i].data)}", massivTauTime)

In [ ]:
from scipy import integrate

sourceFunction = data.ray.source_function

mu = [0.0469101,0.230765,0.500000,0.769235,0.953090]
weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]
massivTauTime = np.load("massivTauTime-102.npy")
print(np.round(wave[102].data))

# len(data.ray.x)
# intensityMu =[]
for m in range(5):
    intensityMu = []

    for x1 in range(50):
        
        tmpMu = 0
        # for i in range(len(massivTau[y])):
        # x = np.log(massivTau[y])
        xx = massivTauTime[x1]
       
        # x1 = np.log10(massivTau[y])
        # x = np.log(x)
        # tmpMu = integrate.trapz(sourceFunction[0,0,:,y].data * np.exp(-x[:]/mu), x)
        # tmpMu = integrate.trapz(sourceFunction[120,0,:,y].data * np.exp(-x[:]/mu[m]), x)     
        ee = np.exp(-x[102]/mu[m])
        tmpMu = integrate.simps(sourceFunction[x1,0,:,102].data * ee, xx)
        # tmpMu = np.power(tmpMu, 10)
        
        intensityMu.append(1/mu[m] * tmpMu)
        # intensityMu.append(tmpMu)
    np.save(f"intensityMu-{m+1}.npy", intensityMu)

    print(f"Всего элементоу: {len(intensityMu)}")
    print(f"Перший элемент: {intensityMu[0]}")
    print(f"Второй элемент: {intensityMu[1]}")
    # print(f"Последний элемент: {intensityMu[1643]}")

# ---

# fig, ax = plt.subplots()

# ax.plot(data.atmos.height_scale[2,0]/1e6,sourceFunction[2,0,:,0],label='0.2 sec')
#ax.plot(np.log10(massivTau[792]),sourceFunction[22,0,:,0],label='0.2 sec')


In [ ]:
%matplotlib widget

wave = data.ray.wavelength
# ImuLoaded = np.load("IntensityMu12.npy")

ImuLoadedT1 = np.load("IntensityMu-1.npy")
ImuLoadedT2 = np.load("IntensityMu-2.npy")
ImuLoadedT3 = np.load("IntensityMu-3.npy")
ImuLoadedT4 = np.load("IntensityMu-4.npy")
ImuLoadedT5 = np.load("IntensityMu-5.npy")
# # print(ImuLoaded1.shape)
fig, ax=plt.subplots()
# #np.load("Flux.npy")



# ax.plot(wave[indices], ImuLoaded,'k--',label=r'$\mu$1')

# ax.plot(wave[indices], ImuLoaded5,'r-',label=r'$\mu$0.95')
# ax.plot(wave[indices],data.ray.intensity[12,0,indices],'g-.',label='1.2 sec')

ax.plot((data.ray.x-10)/10, ImuLoadedT1,'k--',label=r'$\mu$1')
ax.plot((data.ray.x-10)/10, ImuLoadedT2,'k-',label=r'$\mu$2')
ax.plot((data.ray.x-10)/10, ImuLoadedT3,'g-',label=r'$\mu$3')
ax.plot((data.ray.x-10)/10, ImuLoadedT4,'r-',label=r'$\mu$4')
ax.plot((data.ray.x-10)/10, ImuLoadedT5,'b-',label=r'$\mu$5')

ax.legend()
plt.xlabel(r'Time, s')
plt.ylabel(r'Intensity:($W/(m^2*Hz*sr)$')
#ax.set_yscale('log')
#plt.axis([20, 4000, 0, 8e-8])
plt.savefig("Intencity_.svg", format="svg")

In [ ]:
fluxesImuT =[]
weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]
mu = [0.0469101,0.230765,0.500000,0.769235,0.953090]
ImuLoadedT1 = np.load("IntensityMu-1.npy")
ImuLoadedT2 = np.load("IntensityMu-2.npy")
ImuLoadedT3 = np.load("IntensityMu-3.npy")
ImuLoadedT4 = np.load("IntensityMu-4.npy")
ImuLoadedT5 = np.load("IntensityMu-5.npy")
ImuloadedT = []
ImuloadedT.append(ImuLoadedT1)
ImuloadedT.append(ImuLoadedT2)
ImuloadedT.append(ImuLoadedT3)
ImuloadedT.append(ImuLoadedT4)
ImuloadedT.append(ImuLoadedT5)
#print(len(Imuloaded))
for i in range(len(data.ray.x)):
    imu = 0
    for i5 in range(len(weightMu)):
        imu += ImuloadedT[i5][i]*weightMu[i5]*mu[i5]
    fluxesImuT.append(2*np.pi*imu)
print(len(fluxesImuT))

In [ ]:
np.save("fluxesImuT102.npy", fluxesImuT)

In [ ]:
%matplotlib widget

fluxesImuT1 = np.load("fluxesImuT97.npy")
fluxesImuT2 = np.load("fluxesImuT102.npy")




fig, ax=plt.subplots()
#np.load("Flux.npy")

wave = data.ray.wavelength

ax.plot((data.ray.x-10)/10, fluxesImuT1,'k--',label='$\lambda$=262 nm')

ax.plot((data.ray.x-10)/10, fluxesImuT2,'g--',label='$\lambda$=279 nm')


ax.legend()
plt.xlabel(r'Time, s')
plt.ylabel(r'Flux ($W/(m^2*Hz)$')
#ax.set_yscale('log')
#plt.axis([20, 4000, 0, 8e-8])
plt.savefig("FluxperTime_.svg", format="svg")
# ax.plot(wave[indices], ImuLoaded5)
# plt.xlabel("Wavelenght, nm")
# plt.ylabel("Flux, W/cm^2")
#plt.xlim(200,1000)
# plt.axis([2000, 10000, 0, 1e8])
#plt.axis([200, 1000,0, 1e-7])
